# Курсовая работа 

- [Часть первая](#Часть-первая)


## Часть первая


Перед вами стоит задача – подготовить аналитический отчет для HR-отдела. На основании проведенной аналитики предполагается составить рекомендации для отдела кадров по стратегии набора персонала, а также по взаимодействию с уже имеющимися сотрудниками.
<br><br> В базе данных лежит набор таблиц, которые содержат данные о сотрудниках вымышленной компании.
Сделайте обзор штата сотрудников компании. Составьте набор предметов исследования, а затем проверьте их на данных. Вся аналитика должна быть выполена с помощью SQL. <br><br>

Примеры гипотез:
1. Есть зависимость между `perfomance score` и тем, под чьим руководством работает сотрудник.
2. Есть зависимость между продолжительностью работы в компании и семейным положением сотрудника.
2. Есть зависимость между продолжительностью работы в компании и возрастом сотрудника.

<br><br>
Параметры для подключения следующие: хост – `dsstudents.skillbox.ru`, порт – `5432`, имя базы данных – `human_resources`, пользователь – `readonly`, пароль – `6hajV34RTQfmxhS`. Таблицы, доступные для анализа, – `hr_dataset`, `production_staff`, `recruting_costs`, `salary_grid`.

In [1]:
### YOUR CODE HERE ###
import psycopg2
import pandas as pd
import numpy as np 

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "human_resources",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}

conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

# name of tables from database
sql_str = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
cursor.execute(sql_str)
tables_data = [a for a in cursor.fetchall()]
conn.commit()

print("Какие таблицы имеем в БД: %s" % tables_data)

Какие таблицы имеем в БД: [('hr_dataset',), ('production_staff',), ('male',), ('female',), ('recruiting_costs',), ('salary_grid',), ('numb_of_hire',), ('numb_of_terms',), ('show_functions',), ('share_employee_source',)]


In [2]:
# count of rows 

table_names = [i[0] for i in tables_data]
for table in table_names:
    sql_str = "SELECT COUNT(*) FROM public.{};".format(table)
    cursor.execute(sql_str)
    row_count = [a for a in cursor.fetchall()][0][0]
    print("table {}: {}".format(table, row_count))

table hr_dataset: 310
table production_staff: 256
table male: 5
table female: 6
table recruiting_costs: 22
table salary_grid: 12
table show_functions: 0
table share_employee_source: 23


In [3]:
# look on tables: 
# first
df_hr = pd.read_sql_query("SELECT * FROM public.hr_dataset LIMIT 5;", conn)
df_hr

,id,Employee Name,Employee Number,marriedid,maritalstatusid,genderid,empstatus_id,deptid,perf_scoreid,age,...,Date of Hire,Days Employed,Date of Termination,Reason For Term,Employment Status,department,position,Manager Name,Employee Source,Performance Score
0,1,"Brown, Mia",1103024456,1,1,0,1,1,3,30,...,2008-10-27,3317,None,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Diversity Job Fair,Fully Meets
1,2,"LaRotonda, William",1106026572,0,2,1,1,1,3,34,...,2014-01-06,1420,None,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Website Banner Ads,Fully Meets
2,3,"Steans, Tyrone",1302053333,0,0,1,1,1,3,31,...,2014-09-29,1154,None,N/A - still employed,Active,Admin Offices,Accountant I,Brandon R. LeBlanc,Internet Search,Fully Meets
3,4,"Howard, Estelle",1211050782,1,1,0,1,1,9,32,...,2015-02-16,58,2015-04-15,N/A - still employed,Active,Admin Offices,Administrative Assistant,Brandon R. LeBlanc,Pay Per Click - Google,N/A- too early to review
4,5,"Singh, Nan",1307059817,0,0,0,1,1,9,30,...,2015-05-01,940,None,N/A - still employed,Active,Admin Offices,Administrative Assistant,Brandon R. LeBlanc,Website Banner Ads,N/A- too early to review


In [4]:
# what is not visible 
df_hr.iloc[:, 10:19]

,Pay Rate,state,zip,dob,sex,maritaldesc,citizendesc,Hispanic/Latino,racedesc
0,28.50,MA,1450,1987-11-24,Female,Married,US Citizen,No,Black or African American
1,23.00,MA,1460,1984-04-26,Male,Divorced,US Citizen,No,Black or African American
2,29.00,MA,2703,1986-09-01,Male,Single,US Citizen,No,White
3,21.50,MA,2170,1985-09-16,Female,Married,US Citizen,No,White
4,16.56,MA,2330,1988-05-19,Female,Single,US Citizen,No,White


In [6]:
# next 
df_staff = pd.read_sql_query("SELECT * FROM public.production_staff LIMIT 5;", conn)
df_staff

,id,Employee Name,Race Desc,Date of Hire,TermDate,Reason for Term,Employment Status,Department,Position,Pay,Manager Name,Performance Score,Abutments/Hour Wk 1,Abutments/Hour Wk 2,Daily Error Rate,90-day Complaints
0,1,"Albert, Michael",White,2011-08-01,None,N/A - still employed,Active,Production,Production Manager,$54.50,Elisa Bramante,Fully Meets,0,0,0,0
1,2,"Bozzi, Charles",Asian,2013-09-30,2014-08-07,retiring,Voluntarily Terminated,Production,Production Manager,$50.50,Elisa Bramante,Fully Meets,0,0,0,0
2,3,"Butler, Webster L",White,2016-01-28,None,N/A - still employed,Active,Production,Production Manager,$55.00,Elisa Bramante,Exceeds,0,0,0,0
3,4,"Dunn, Amy",White,2014-09-18,None,N/A - still employed,Active,Production,Production Manager,$51.00,Elisa Bramante,Fully Meets,0,0,0,0
4,5,"Gray, Elijiah",White,2015-06-02,None,N/A - still employed,Active,Production,Production Manager,$54.00,Elisa Bramante,Fully Meets,0,0,0,0


In [7]:
# next 
df_recr = pd.read_sql_query("SELECT * FROM public.recruiting_costs;", conn)
df_recr

,id,Employment Source,January,February,March,April,May,June,July,August,September,October,November,December,Total
0,1,Billboard,520,520,520,520,0,0,612,612,729,749,910,500,6192
1,2,Careerbuilder,410,410,410,820,820,410,410,820,820,1230,820,410,7790
2,3,Company Intranet - Partner,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,Diversity Job Fair,0,5129,0,0,0,0,0,4892,0,0,0,0,10021
4,5,Employee Referral,0,0,0,0,0,0,0,0,0,0,0,0,0
5,6,Glassdoor,0,0,0,0,0,0,0,0,0,0,0,0,0
6,7,Information Session,0,0,0,0,0,0,0,0,0,0,0,0,0
7,8,Internet Search,0,0,0,0,0,0,0,0,0,0,0,0,0
8,9,MBTA ads,640,640,640,640,640,640,640,1300,1300,1300,1300,1300,10980
9,10,Monster.com,500,500,500,440,500,500,440,500,440,440,500,500,5760


In [8]:
# and last table 
df_sal = pd.read_sql_query("SELECT * FROM public.salary_grid;", conn)
df_sal

,id,Position,Salary Min,Salary Mid,Salary Max,Hourly Min,Hourly Mid,Hourly Max
0,1,Administrative Assistant,30000,40000,50000,14.42,19.23,24.04
1,2,Sr. Administrative Assistant,35000,45000,55000,16.83,21.63,26.44
2,3,Accountant I,42274,51425,62299,20.32,24.72,29.95
3,4,Accountant II,50490,62158,74658,24.27,29.88,35.89
4,5,Sr. Accountant,63264,76988,92454,30.42,37.01,44.45
5,6,Network Engineer,50845,66850,88279,24.44,32.14,42.44
6,7,Sr. Network Engineer,79428,99458,120451,38.19,47.82,57.91
7,8,Database Administrator,50569,68306,93312,24.31,32.84,44.86
8,9,Sr. DBA,92863,116007,139170,44.65,55.77,66.91
9,10,Production Technician I,30000,40000,50000,14.42,19.23,24.04


In [9]:
# next 
df_male = pd.read_sql_query("SELECT * FROM public.male;", conn)
df_male

,department,num_sex
0,Admin Offices,4
1,IT/IS,28
2,Production,81
3,Sales,16
4,Software Engineering,4


In [11]:
# next 
df_female = pd.read_sql_query("SELECT * FROM public.female;", conn)
df_female

,department,num_sex
0,Admin Offices,6
1,Executive Office,1
2,IT/IS,22
3,Production,127
4,Sales,15
5,Software Engineering,6


In [12]:
# next 
df_emp_source = pd.read_sql_query("SELECT * FROM public.share_employee_source;", conn)
df_emp_source

,employee_source,share_source
0,Company Intranet - Partner,0.0
1,Professional Society,6.0
2,On-line Web application,0.0
3,Billboard,5.0
4,Pay Per Click,0.0
5,MBTA ads,5.0
6,Newspager/Magazine,5.0
7,Search Engine - Google Bing Yahoo,8.0
8,Pay Per Click - Google,6.0
9,Internet Search,1.0


In [3]:
# Create a function
def sql_simple(sql_str):
    cursor.execute(sql_str)
    list_simple = []
    for row in cursor:
        list_simple.append(row[0])
    print(*list_simple, sep=", ")

In [14]:
# let's look on content of columns table hr_dataset

sql_str1 = "SELECT DISTINCT age FROM public.hr_dataset ORDER BY age;"
sql_simple(sql_str1)

25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 58, 59, 62, 63, 65, 66, 67


In [15]:
sql_str2 = "SELECT DISTINCT state FROM public.hr_dataset;"
sql_simple(sql_str2)

CA, NH, OR, ND, TX, NV, KY, OH, NY, IN, FL, ME, NC, CT, RI, GA, ID, PA, MT, TN, WA, MA, AL, UT, CO, VT, VA, AZ


In [10]:
sql_str3 = "SELECT DISTINCT state FROM public.hr_dataset;"
sql_simple(sql_str3)

CA, NH, OR, ND, TX, NV, KY, OH, NY, IN, FL, ME, NC, CT, RI, GA, ID, PA, MT, TN, WA, MA, AL, UT, CO, VT, VA, AZ


In [11]:
sql_str4 = "SELECT DISTINCT maritaldesc FROM public.hr_dataset;"
sql_simple(sql_str4)

Widowed, Married, Separated, Single, Divorced


In [12]:
sql_str5 = "SELECT DISTINCT citizendesc FROM public.hr_dataset;"
sql_simple(sql_str5)

Eligible NonCitizen, Non-Citizen, US Citizen


In [13]:
sql_str6 = "SELECT DISTINCT racedesc FROM public.hr_dataset;"
sql_simple(sql_str6)

American Indian or Alaska Native, Hispanic, Two or more races, Asian, White, Black or African American


In [14]:
sql_str7 = '''SELECT DISTINCT "Reason For Term" FROM public.hr_dataset;'''
sql_simple(sql_str7)

N/A - Has not started yet, attendance, more money, medical issues, maternity leave - did not return, return to school, relocation out of area, performance, gross misconduct, military, hours, career change, unhappy, retiring, Another position, no-call, no-show, N/A - still employed


In [15]:
sql_str8 = '''SELECT DISTINCT "Employment Status" FROM public.hr_dataset;'''
sql_simple(sql_str8)

Future Start, Voluntarily Terminated, Active, Terminated for Cause, Leave of Absence


In [16]:
sql_str9 = "SELECT DISTINCT department FROM public.hr_dataset;"
sql_simple(sql_str9)

Production       , Admin Offices, IT/IS, Executive Office, Software Engineering, Sales


In [17]:
sql_str10 = "SELECT DISTINCT position FROM public.hr_dataset;"
sql_simple(sql_str10)

IT Support, Database Administrator, Software Engineering Manager, Production Manager, Senior BI Developer, Shared Services Manager, BI Developer, CIO, Sales Manager, Sr. DBA, Network Engineer, Area Sales Manager, Administrative Assistant, Data Architect, IT Manager - DB, IT Manager - Infra, IT Director, Production Technician I, BI Director, Software Engineer, Sr. Network Engineer, Director of Sales, President & CEO, Sr. Accountant, Production Technician II, IT Manager - Support, Director of Operations, Accountant I


In [18]:
sql_str11 = '''SELECT DISTINCT "Manager Name" FROM public.hr_dataset;'''
sql_simple(sql_str11)

Brian Champaigne, Michael Albert, Simon Roup, John Smith, Amy Dunn, Brannon Miller, Webster Butler, Brandon R. LeBlanc, Janet King, Lynn Daneault, Elijiah Gray, Alex Sweetwater, David Stanley, Peter Monroe, Eric Dougall, Jennifer Zamora, Board of Directors, Kelley Spirea, Kissy Sullivan, Ketsia Liebig, Debra Houlihan


In [19]:
sql_str12 = '''SELECT DISTINCT "Performance Score" FROM public.hr_dataset;'''
sql_simple(sql_str12)

Exceptional, 90-day meets, N/A- too early to review, Fully Meets, Exceeds, PIP, Needs Improvement


In [20]:
# let's look on content of columns table production_staff

sql_str13 = '''SELECT DISTINCT "Reason for Term" FROM public.production_staff;'''
sql_simple(sql_str13)

None, N/A - Has not started yet, attendance, more money, medical issues, maternity leave - did not return, return to school, performance, relocation out of area, gross misconduct, military, hours, career change, unhappy, retiring, Another position, no-call, no-show, N/A - still employed


In [21]:
sql_str14 = '''SELECT DISTINCT "Employment Status" FROM public.production_staff;'''
sql_simple(sql_str14)

Leave of Absence, None, Future Start, Voluntarily Terminated, Active, Terminated for Cause


In [22]:
sql_str15 = '''SELECT DISTINCT "Daily Error Rate" FROM public.production_staff;'''
sql_simple(sql_str15)

4, 2, None, 0, 1, 3


In [23]:
sql_str16 = '''SELECT DISTINCT "90-day Complaints" FROM public.production_staff;'''
sql_simple(sql_str16)

4, 2, None, 0, 1, 3


In [24]:
sql_str17 = '''SELECT DISTINCT "Department" FROM public.production_staff;'''
sql_simple(sql_str17)

None, Production       


In [25]:
sql_str18 = '''SELECT DISTINCT "Position" FROM public.production_staff;'''
sql_simple(sql_str18)

Production Technician I, None, Production Manager, Production Technician II


In [26]:
sql_str19 = '''SELECT DISTINCT "Manager Name" FROM public.production_staff;'''
sql_simple(sql_str19)

Ketsia Liebig, Amy Dunn, None, Brannon Miller, Webster Butler, Michael Albert, Elijiah Gray, David Stanley, Elisa Bramante, Kelley Spirea, Kissy Sullivan


In [77]:
sql_str20 = '''SELECT DISTINCT "Performance Score" FROM public.production_staff;'''
sql_simple(sql_str20)

Needs Improvement, None, Exceptional, 90-day meets, N/A- too early to review, Fully Meets, Exceeds, PIP


In [76]:
# Легко заметить, что:
# 1. таблица hr_dataset связана с таблицей salary_grid через колонку position 
# 2. таблица hr_dataset связана с таблицей recruiting_costs через колонку employee source 
# 3. таблица production_staff связана с таблицей salary_grid через колонку position 

In [ ]:
# I. LET'S EXPLORE THE TABLE 'HR_DATASET'!



In [4]:
# Create a function № 2
def sql_medium(sql_str):
    cursor.execute(sql_str)
    list_medium = []
    for row in cursor:
        list_medium.append(row[0])
    array_dict = {}.fromkeys(list_medium, 0)
    for elem in list_medium:
        array_dict[elem] += 1
    print(array_dict)

In [17]:
# Is there a relationship between performance score and who the employee is working under?
# - Fully Meets
sql_str21 = """
    SELECT "Manager Name" FROM public.hr_dataset 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str21)

{'Brandon R. LeBlanc': 4, 'Janet King': 13, 'Board of Directors': 2, 'Simon Roup': 6, 'Jennifer Zamora': 4, 'Eric Dougall': 3, 'Peter Monroe': 6, 'Elijiah Gray': 13, 'Brannon Miller': 8, 'Kelley Spirea': 15, 'Michael Albert': 9, 'Webster Butler': 11, 'Amy Dunn': 11, 'Ketsia Liebig': 14, 'David Stanley': 15, 'Kissy Sullivan': 14, 'Lynn Daneault': 8, 'John Smith': 11, 'Debra Houlihan': 2, 'Alex Sweetwater': 4, 'Brian Champaigne': 8}


In [11]:
# Is there a relationship between performance score and who the employee is working under?
# - Exceeds
sql_str22 = """
    SELECT "Manager Name" FROM public.hr_dataset 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str22)

{'Jennifer Zamora': 1, 'Eric Dougall': 1, 'Janet King': 3, 'Michael Albert': 2, 'Brannon Miller': 5, 'Amy Dunn': 2, 'Webster Butler': 2, 'Kelley Spirea': 3, 'David Stanley': 1, 'Kissy Sullivan': 1, 'Ketsia Liebig': 2, 'Elijiah Gray': 2, 'Lynn Daneault': 2, 'Alex Sweetwater': 1}


In [12]:
# Is there a relationship between performance score and who the employee is working under?
# - Needs Improvement
sql_str23 = """
    SELECT "Manager Name" FROM public.hr_dataset 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str23)

{'Jennifer Zamora': 1, 'Janet King': 2, 'Kissy Sullivan': 1, 'Brannon Miller': 1, 'Michael Albert': 3, 'Webster Butler': 2, 'Ketsia Liebig': 1, 'Elijiah Gray': 1, 'John Smith': 1, 'Debra Houlihan': 1, 'Alex Sweetwater': 1}


In [13]:
# Is there a relationship between performance score and the marital status of the employee?
# - Fully Meets
sql_str24 = """
    SELECT maritaldesc FROM public.hr_dataset 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str24)

{'Married': 73, 'Divorced': 16, 'Single': 81, 'Separated': 8, 'Widowed': 3}


In [14]:
# Is there a relationship between performance score and the marital status of the employee?
# - Exceeds
sql_str25 = """
    SELECT maritaldesc FROM public.hr_dataset 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str25)

{'Single': 12, 'Married': 10, 'Divorced': 4, 'Separated': 2}


In [34]:
# Is there a relationship between performance score and the marital status of the employee?
# - Needs Improvement
sql_str26 = """
    SELECT maritaldesc FROM public.hr_dataset 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str26)

{'Married': 6, 'Divorced': 2, 'Single': 7}


In [35]:
# Is there a relationship between performance score and the age of the employee?
# - Fully Meets
sql_str27 = """
    SELECT age FROM public.hr_dataset 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str27)

{30: 10, 34: 11, 31: 9, 33: 10, 39: 14, 63: 3, 32: 7, 46: 3, 36: 6, 45: 7, 54: 4, 49: 5, 37: 3, 43: 8, 53: 3, 66: 1, 48: 5, 44: 4, 41: 5, 42: 5, 40: 6, 62: 1, 29: 9, 47: 5, 59: 1, 27: 3, 35: 7, 67: 2, 26: 1, 38: 6, 25: 2, 65: 2, 52: 3, 50: 1, 56: 1, 51: 2, 55: 1, 28: 5}


In [36]:
# Is there a relationship between performance score and the age of the employee?
# - Exceeds
sql_str28 = """
    SELECT age FROM public.hr_dataset 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str28)

{47: 1, 29: 2, 35: 1, 34: 2, 36: 2, 28: 1, 37: 1, 38: 1, 48: 1, 44: 1, 58: 1, 45: 3, 51: 1, 41: 1, 31: 3, 32: 1, 52: 1, 54: 1, 59: 1, 50: 1, 39: 1}


In [37]:
# Is there a relationship between performance score and the age of the employee?
# - Needs Improvement
sql_str29 = """
    SELECT age FROM public.hr_dataset 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str29)

{31: 2, 41: 1, 42: 2, 38: 2, 39: 1, 34: 1, 43: 1, 30: 1, 33: 2, 37: 1, 27: 1}


In [38]:
# Is there a relationship between performance score and the migration status of the employee?
# - Fully Meets
sql_str30 = """
    SELECT citizendesc FROM public.hr_dataset 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str30)

{'US Citizen': 170, 'Eligible NonCitizen': 7, 'Non-Citizen': 4}


In [39]:
# Is there a relationship between performance score and the migration status of the employee?
# - Exceeds
sql_str31 = """
    SELECT citizendesc FROM public.hr_dataset 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str31)

{'US Citizen': 28}


In [40]:
# Is there a relationship between performance score and the migration status of the employee?
# - Needs Improvement
sql_str32 = """
    SELECT citizendesc FROM public.hr_dataset 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str32)

{'Eligible NonCitizen': 1, 'US Citizen': 14}


In [41]:
# Is there a relationship between performance score and the race of the employee?
# - Fully Meets
sql_str33 = """
    SELECT racedesc FROM public.hr_dataset 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str33)

{'Black or African American': 37, 'White': 110, 'Asian': 20, 'Two or more races': 10, 'Hispanic': 2, 'American Indian or Alaska Native': 2}


In [42]:
# Is there a relationship between performance score and the race of the employee?
# - Exceeds
sql_str34 = """
    SELECT racedesc FROM public.hr_dataset 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str34)

{'White': 18, 'Two or more races': 3, 'Black or African American': 2, 'American Indian or Alaska Native': 2, 'Asian': 3}


In [43]:
# Is there a relationship between performance score and the race of the employee?
# - Needs Improvement
sql_str35 = """
    SELECT racedesc FROM public.hr_dataset 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str35)

{'Hispanic': 1, 'Asian': 1, 'White': 5, 'Black or African American': 7, 'Two or more races': 1}


In [44]:
# Is there a relationship between performance score and the department of the employee?
# - Fully Meets
sql_str36 = """
    SELECT department FROM public.hr_dataset 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str36)

{'Admin Offices': 7, 'Executive Office': 1, 'IT/IS': 26, 'Production       ': 120, 'Sales': 22, 'Software Engineering': 5}


In [45]:
# Is there a relationship between performance score and the department of the employee?
# - Exceeds
sql_str37 = """
    SELECT department FROM public.hr_dataset 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str37)

{'IT/IS': 2, 'Production       ': 23, 'Sales': 2, 'Software Engineering': 1}


In [46]:
# Is there a relationship between performance score and the department of the employee?
# - Needs Improvement
sql_str38 = """
    SELECT department FROM public.hr_dataset 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str38)

{'IT/IS': 1, 'Production       ': 11, 'Sales': 2, 'Software Engineering': 1}


In [47]:
# Is there a relationship between performance score and the position of the employee?
# - Fully Meets
sql_str39 = """
    SELECT position FROM public.hr_dataset 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str39)

{'Accountant I': 3, 'Administrative Assistant': 1, 'Shared Services Manager': 2, 'Sr. Accountant': 1, 'President & CEO': 1, 'Database Administrator': 4, 'IT Manager - DB': 2, 'IT Support': 3, 'Network Engineer': 4, 'Sr. DBA': 2, 'Sr. Network Engineer': 2, 'Production Manager': 10, 'Production Technician I': 78, 'Production Technician II': 32, 'Area Sales Manager': 19, 'Director of Sales': 1, 'Sales Manager': 2, 'Software Engineer': 4, 'Software Engineering Manager': 1, 'BI Director': 1, 'Senior BI Developer': 3, 'BI Developer': 4, 'Data Architect': 1}


In [48]:
# Is there a relationship between performance score and the position of the employee?
# - Exceeds
sql_str40 = """
    SELECT position FROM public.hr_dataset 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str40)

{'IT Manager - Support': 1, 'IT Support': 1, 'Director of Operations': 1, 'Production Manager': 2, 'Production Technician I': 14, 'Production Technician II': 6, 'Area Sales Manager': 2, 'Software Engineer': 1}


In [49]:
# Is there a relationship between performance score and the position of the employee?
# - Needs Improvement
sql_str41 = """
    SELECT position FROM public.hr_dataset 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str41)

{'IT Manager - Infra': 1, 'Production Manager': 2, 'Production Technician I': 8, 'Production Technician II': 1, 'Area Sales Manager': 1, 'Sales Manager': 1, 'Software Engineer': 1}


In [50]:
# Is there a relationship between performance score and the employee source?
# - Fully Meets
sql_str42 = """
    SELECT "Employee Source" FROM public.hr_dataset 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str42)

{'Diversity Job Fair': 14, 'Website Banner Ads': 6, 'Internet Search': 4, 'Monster.com': 15, 'Other': 3, 'Pay Per Click - Google': 12, 'Search Engine - Google Bing Yahoo': 19, 'Glassdoor': 9, 'Employee Referral': 16, 'Professional Society': 9, 'Vendor Referral': 9, 'Information Session': 2, 'Company Intranet - Partner': 1, 'Billboard': 10, 'Social Networks - Facebook Twitter etc': 6, 'On-line Web application': 1, 'MBTA ads': 9, 'Word of Mouth': 8, 'Newspager/Magazine': 10, 'On-campus Recruiting': 9, 'Careerbuilder': 1, 'Indeed': 8}


In [51]:
# Is there a relationship between performance score and the employee source?
# - Exceeds
sql_str43 = """
    SELECT "Employee Source" FROM public.hr_dataset 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str43)

{'Professional Society': 4, 'Diversity Job Fair': 5, 'Other': 2, 'Pay Per Click - Google': 3, 'Website Banner Ads': 1, 'MBTA ads': 3, 'Employee Referral': 2, 'Monster.com': 2, 'On-campus Recruiting': 1, 'Search Engine - Google Bing Yahoo': 1, 'Information Session': 1, 'Social Networks - Facebook Twitter etc': 1, 'Glassdoor': 1, 'Billboard': 1}


In [52]:
# Is there a relationship between performance score and the employee source?
# - Needs Improvement
sql_str44 = """
    SELECT "Employee Source" FROM public.hr_dataset 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str44)
#########################################################

{'Diversity Job Fair': 3, 'Other': 1, 'Monster.com': 3, 'MBTA ads': 2, 'Pay Per Click - Google': 1, 'Word of Mouth': 1, 'Search Engine - Google Bing Yahoo': 1, 'Glassdoor': 1, 'Internet Search': 1, 'Billboard': 1}


In [53]:
# Is there a relationship between the length of work in the company 
# and the marital status of the employee? 10-most length. 
sql_str45 = """
    SELECT maritaldesc FROM public.hr_dataset 
    ORDER BY "Days Employed" DESC LIMIT 10;
"""
sql_medium(sql_str45)

{'Separated': 1, 'Single': 4, 'Married': 5}


In [54]:
# Is there a relationship between the length of work in the company 
# and the age of the employee? 10-most length. 
sql_str46 = """
    SELECT age FROM public.hr_dataset 
    ORDER BY "Days Employed" DESC LIMIT 10;
"""
sql_medium(sql_str46)

{50: 1, 36: 1, 29: 1, 30: 1, 35: 1, 39: 1, 40: 1, 48: 1, 38: 1, 44: 1}


In [55]:
# Is there a relationship between the length of work in the company and the migration status?
# 10-most length.
sql_str47 = """
    SELECT citizendesc FROM public.hr_dataset 
    ORDER BY "Days Employed" DESC LIMIT 10;
"""
sql_medium(sql_str47)

{'US Citizen': 10}


In [56]:
# Is there a relationship between the length of work in the company and race?
# 10-most length.
sql_str48 = """
    SELECT racedesc FROM public.hr_dataset 
    ORDER BY "Days Employed" DESC LIMIT 10;
"""
sql_medium(sql_str48)

{'White': 5, 'Black or African American': 4, 'Two or more races': 1}


In [57]:
# Is there a relationship between the length of work in the company and department?
# 10-most length.
sql_str49 = """
    SELECT department FROM public.hr_dataset 
    ORDER BY "Days Employed" DESC LIMIT 10;
"""
sql_medium(sql_str49)

{'Sales': 1, 'Production       ': 6, 'Admin Offices': 2, 'IT/IS': 1}


In [58]:
# Is there a relationship between the length of work in the company and position?
# 10-most length.
sql_str50 = """
    SELECT position FROM public.hr_dataset 
    ORDER BY "Days Employed" DESC LIMIT 10;
"""
sql_medium(sql_str50)

{'Area Sales Manager': 1, 'Production Technician I': 4, 'Accountant I': 1, 'Director of Operations': 1, 'Sr. Accountant': 1, 'Production Manager': 1, 'CIO': 1}


In [73]:
# Is there a relationship between the length of work in the company and position?
# 10-most length.
sql_str51 = """
    SELECT "Employee Source" FROM public.hr_dataset 
    ORDER BY "Days Employed" DESC LIMIT 10;
"""
sql_medium(sql_str51)

{'Billboard': 3, 'Search Engine - Google Bing Yahoo': 2, 'Diversity Job Fair': 2, 'Other': 2, 'Employee Referral': 1}


In [79]:
# Is there a relationship between the length of work in the company and manager name?
# 10-most length.
sql_str52 = """
    SELECT "Manager Name" FROM public.hr_dataset 
    ORDER BY "Days Employed" DESC LIMIT 10;
"""
sql_medium(sql_str52)

{'Lynn Daneault': 1, 'David Stanley': 1, 'Elijiah Gray': 1, 'Brandon R. LeBlanc': 1, 'Janet King': 3, 'Board of Directors': 1, 'Brannon Miller': 1, 'Ketsia Liebig': 1}


In [4]:
# Number of employees who have the status 'N/A - Has not started yet' in "Reason For Term".
sql_str53 = """
    SELECT COUNT (*) FROM public.hr_dataset 
    WHERE "Reason For Term" = 'N/A - Has not started yet';
"""
cursor.execute(sql_str53)
row_count1 = [a for a in cursor.fetchall()][0][0]
print(row_count1)

11


In [5]:
# Number of employees who have the status 'N/A - still employed' in "Reason For Term".
sql_str54 = """
    SELECT COUNT (*) FROM public.hr_dataset 
    WHERE "Reason For Term" = 'N/A - still employed';
"""
cursor.execute(sql_str54)
row_count2 = [a for a in cursor.fetchall()][0][0]
print(row_count2)

197


In [89]:
# Is there a relationship between termination and the manager name?
sql_str55 = """
    SELECT "Manager Name" FROM public.hr_dataset 
    WHERE "Reason For Term" NOT IN ('N/A - Has not started yet', 
    'N/A - still employed');
"""
sql_medium(sql_str55)

{'Brandon R. LeBlanc': 1, 'Janet King': 6, 'Simon Roup': 8, 'Jennifer Zamora': 1, 'Peter Monroe': 1, 'Webster Butler': 13, 'David Stanley': 6, 'Kissy Sullivan': 12, 'Michael Albert': 9, 'Amy Dunn': 13, 'Elijiah Gray': 8, 'Kelley Spirea': 6, 'Brannon Miller': 6, 'Ketsia Liebig': 5, 'John Smith': 3, 'Debra Houlihan': 1, 'Alex Sweetwater': 3}


In [90]:
# Is there a relationship between termination and the maritaldesc?
sql_str56 = """
    SELECT maritaldesc FROM public.hr_dataset 
    WHERE "Reason For Term" NOT IN ('N/A - Has not started yet', 
    'N/A - still employed');
"""
sql_medium(sql_str56)

{'Married': 45, 'Single': 36, 'Divorced': 16, 'Widowed': 4, 'Separated': 1}


In [91]:
# Is there a relationship between termination and the age?
sql_str57 = """
    SELECT age FROM public.hr_dataset 
    WHERE "Reason For Term" NOT IN ('N/A - Has not started yet', 
    'N/A - still employed');
"""
sql_medium(sql_str57)

{30: 6, 33: 6, 32: 4, 46: 2, 31: 8, 54: 2, 43: 4, 53: 3, 45: 7, 48: 3, 41: 4, 62: 1, 28: 3, 44: 4, 40: 2, 27: 1, 34: 7, 38: 2, 26: 1, 29: 3, 63: 1, 35: 3, 65: 2, 58: 1, 47: 2, 39: 4, 36: 5, 37: 3, 50: 1, 52: 1, 51: 1, 67: 1, 66: 1, 42: 1, 59: 1, 49: 1}


In [92]:
# Is there a relationship between termination and the citizendesc?
sql_str58 = """
    SELECT citizendesc FROM public.hr_dataset 
    WHERE "Reason For Term" NOT IN ('N/A - Has not started yet', 
    'N/A - still employed');
"""
sql_medium(sql_str58)

{'US Citizen': 94, 'Eligible NonCitizen': 5, 'Non-Citizen': 3}


In [93]:
# Is there a relationship between termination and the racedesc?
sql_str59 = """
    SELECT racedesc FROM public.hr_dataset 
    WHERE "Reason For Term" NOT IN ('N/A - Has not started yet', 
    'N/A - still employed');
"""
sql_medium(sql_str59)

{'Asian': 11, 'Black or African American': 17, 'White': 66, 'Two or more races': 7, 'Hispanic': 1}


In [94]:
# Is there a relationship between termination and the department?
sql_str60 = """
    SELECT department FROM public.hr_dataset 
    WHERE "Reason For Term" NOT IN ('N/A - Has not started yet', 
    'N/A - still employed');
"""
sql_medium(sql_str60)

{'Admin Offices': 2, 'IT/IS': 10, 'Production       ': 83, 'Sales': 4, 'Software Engineering': 3}


In [95]:
# Is there a relationship between termination and the position?
sql_str61 = """
    SELECT position FROM public.hr_dataset 
    WHERE "Reason For Term" NOT IN ('N/A - Has not started yet', 
    'N/A - still employed');
"""
sql_medium(sql_str61)

{'Administrative Assistant': 1, 'Shared Services Manager': 1, 'Database Administrator': 5, 'IT Manager - DB': 1, 'Network Engineer': 1, 'Sr. DBA': 3, 'Production Manager': 5, 'Production Technician I': 52, 'Production Technician II': 26, 'Area Sales Manager': 3, 'Sales Manager': 1, 'Software Engineer': 3}


In [96]:
# Is there a relationship between termination and the Employee Source?
sql_str62 = """
    SELECT "Employee Source" FROM public.hr_dataset 
    WHERE "Reason For Term" NOT IN ('N/A - Has not started yet', 
    'N/A - still employed');
"""
sql_medium(sql_str62)

{'Diversity Job Fair': 16, 'Search Engine - Google Bing Yahoo': 15, 'Glassdoor': 6, 'Employee Referral': 4, 'Vendor Referral': 4, 'Monster.com': 11, 'Company Intranet - Partner': 1, 'Billboard': 5, 'Other': 3, 'Internet Search': 2, 'Word of Mouth': 7, 'On-line Web application': 1, 'MBTA ads': 4, 'Social Networks - Facebook Twitter etc': 8, 'Pay Per Click': 1, 'Professional Society': 3, 'Pay Per Click - Google': 3, 'Information Session': 1, 'Newspager/Magazine': 5, 'Website Banner Ads': 1, 'On-campus Recruiting': 1}


In [75]:
# Is there a relationship between termination and the performance score?
sql_str63 = """
    SELECT "Performance Score" FROM public.hr_dataset 
    WHERE "Reason For Term" NOT IN ('N/A - Has not started yet', 
    'N/A - still employed');
"""
sql_medium(sql_str63)

{'Fully Meets': 56, 'N/A- too early to review': 13, '90-day meets': 13, 'Needs Improvement': 8, 'Exceeds': 8, 'PIP': 4}


In [17]:
# 'HR_DATASET' AND 'SALARY_GRID'


In [15]:
# which managers get the maximum salary for their employees? Top 10 max
sql_sal = """
    SELECT hr_dataset."Manager Name", MAX("Salary Max") as sal_max 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal)
for row in cursor:
    print('manage_name: %s \t sal_max: %s' %row)

manage_name: Simon Roup 	 sal_max: 139170
manage_name: Simon Roup 	 sal_max: 139170
manage_name: Simon Roup 	 sal_max: 139170
manage_name: Simon Roup 	 sal_max: 139170
manage_name: Peter Monroe 	 sal_max: 120451
manage_name: Peter Monroe 	 sal_max: 120451
manage_name: Peter Monroe 	 sal_max: 120451
manage_name: Peter Monroe 	 sal_max: 120451
manage_name: Peter Monroe 	 sal_max: 120451
manage_name: Simon Roup 	 sal_max: 93312


In [7]:
# which managers get the minimum salary for their employees? Top 10 min
sql_sal2 = """
    SELECT hr_dataset."Manager Name", MIN("Salary Min") as sal_min 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal2)
for row in cursor:
    print('manage_name: %s \t sal_min: %s' %row)

manage_name: Brannon Miller 	 sal_min: 30000
manage_name: Elijiah Gray 	 sal_min: 30000
manage_name: Webster Butler 	 sal_min: 30000
manage_name: Webster Butler 	 sal_min: 30000
manage_name: Webster Butler 	 sal_min: 30000
manage_name: David Stanley 	 sal_min: 30000
manage_name: Kissy Sullivan 	 sal_min: 30000
manage_name: David Stanley 	 sal_min: 30000
manage_name: Elijiah Gray 	 sal_min: 30000
manage_name: Brannon Miller 	 sal_min: 30000


In [8]:
# which citizendesc get the maximum salary for their employees? Top 10 max
sql_sal3 = """
    SELECT citizendesc, MAX("Salary Max") as sal_max 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal3)
for row in cursor:
    print('citizendesc: %s \t sal_max: %s' %row)

citizendesc: US Citizen 	 sal_max: 139170
citizendesc: US Citizen 	 sal_max: 139170
citizendesc: US Citizen 	 sal_max: 139170
citizendesc: US Citizen 	 sal_max: 139170
citizendesc: US Citizen 	 sal_max: 120451
citizendesc: US Citizen 	 sal_max: 120451
citizendesc: US Citizen 	 sal_max: 120451
citizendesc: US Citizen 	 sal_max: 120451
citizendesc: US Citizen 	 sal_max: 120451
citizendesc: US Citizen 	 sal_max: 93312


In [9]:
# which citizendesc get the minimum salary for their employees? Top 10 min
sql_sal4 = """
    SELECT citizendesc, MIN("Salary Min") as sal_min 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal4)
for row in cursor:
    print('citizendesc: %s \t sal_min: %s' %row)

citizendesc: US Citizen 	 sal_min: 30000
citizendesc: US Citizen 	 sal_min: 30000
citizendesc: US Citizen 	 sal_min: 30000
citizendesc: US Citizen 	 sal_min: 30000
citizendesc: Non-Citizen 	 sal_min: 30000
citizendesc: US Citizen 	 sal_min: 30000
citizendesc: US Citizen 	 sal_min: 30000
citizendesc: US Citizen 	 sal_min: 30000
citizendesc: US Citizen 	 sal_min: 30000
citizendesc: US Citizen 	 sal_min: 30000


In [10]:
# which maritaldesc get the maximum salary for their employees? Top 10 max
sql_sal5 = """
    SELECT maritaldesc, MAX("Salary Max") as sal_max 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal5)
for row in cursor:
    print('maritaldesc: %s \t sal_max: %s' %row)

maritaldesc: Single 	 sal_max: 139170
maritaldesc: Single 	 sal_max: 139170
maritaldesc: Married 	 sal_max: 139170
maritaldesc: Single 	 sal_max: 139170
maritaldesc: Married 	 sal_max: 120451
maritaldesc: Married 	 sal_max: 120451
maritaldesc: Married 	 sal_max: 120451
maritaldesc: Single 	 sal_max: 120451
maritaldesc: Widowed 	 sal_max: 120451
maritaldesc: Single 	 sal_max: 93312


In [11]:
# which maritaldesc get the minimum salary for their employees? Top 10 min
sql_sal6 = """
    SELECT maritaldesc, MIN("Salary Min") as sal_min 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal6)
for row in cursor:
    print('maritaldesc: %s \t sal_min: %s' %row)

maritaldesc: Married 	 sal_min: 30000
maritaldesc: Single 	 sal_min: 30000
maritaldesc: Married 	 sal_min: 30000
maritaldesc: Divorced 	 sal_min: 30000
maritaldesc: Married 	 sal_min: 30000
maritaldesc: Separated 	 sal_min: 30000
maritaldesc: Divorced 	 sal_min: 30000
maritaldesc: Single 	 sal_min: 30000
maritaldesc: Married 	 sal_min: 30000
maritaldesc: Separated 	 sal_min: 30000


In [12]:
# which age get the maximum salary for their employees? Top 10 max
sql_sal7 = """
    SELECT age, MAX("Salary Max") as sal_max 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal7)
for row in cursor:
    print('age: %s \t sal_max: %s' %row)

age: 53 	 sal_max: 139170
age: 45 	 sal_max: 139170
age: 43 	 sal_max: 139170
age: 31 	 sal_max: 139170
age: 32 	 sal_max: 120451
age: 35 	 sal_max: 120451
age: 66 	 sal_max: 120451
age: 52 	 sal_max: 120451
age: 40 	 sal_max: 120451
age: 32 	 sal_max: 93312


In [13]:
# which age get the minimum salary for their employees? Top 10 min
sql_sal8 = """
    SELECT age, MIN("Salary Min") as sal_min 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal8)
for row in cursor:
    print('age: %s \t sal_min: %s' %row)

age: 49 	 sal_min: 30000
age: 39 	 sal_min: 30000
age: 43 	 sal_min: 30000
age: 28 	 sal_min: 30000
age: 43 	 sal_min: 30000
age: 43 	 sal_min: 30000
age: 47 	 sal_min: 30000
age: 40 	 sal_min: 30000
age: 29 	 sal_min: 30000
age: 51 	 sal_min: 30000


In [14]:
# which racedesc get the maximum salary for their employees? Top 10 max
sql_sal9 = """
    SELECT racedesc, MAX("Salary Max") as sal_max 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal9)
for row in cursor:
    print('racedesc: %s \t sal_max: %s' %row)

racedesc: Black or African American 	 sal_max: 139170
racedesc: White 	 sal_max: 139170
racedesc: White 	 sal_max: 139170
racedesc: Black or African American 	 sal_max: 139170
racedesc: White 	 sal_max: 120451
racedesc: Asian 	 sal_max: 120451
racedesc: White 	 sal_max: 120451
racedesc: White 	 sal_max: 120451
racedesc: Asian 	 sal_max: 120451
racedesc: White 	 sal_max: 93312


In [15]:
# which racedesc get the minimum salary for their employees? Top 10 min
sql_sal10 = """
    SELECT racedesc, MIN("Salary Min") as sal_min 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal10)
for row in cursor:
    print('racedesc: %s \t sal_min: %s' %row)

racedesc: White 	 sal_min: 30000
racedesc: White 	 sal_min: 30000
racedesc: Black or African American 	 sal_min: 30000
racedesc: White 	 sal_min: 30000
racedesc: Two or more races 	 sal_min: 30000
racedesc: White 	 sal_min: 30000
racedesc: White 	 sal_min: 30000
racedesc: White 	 sal_min: 30000
racedesc: Black or African American 	 sal_min: 30000
racedesc: Asian 	 sal_min: 30000


In [16]:
# which department get the maximum salary for their employees? Top 10 max
sql_sal11 = """
    SELECT department, MAX("Salary Max") as sal_max 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal11)
for row in cursor:
    print('department: %s \t sal_max: %s' %row)

department: IT/IS 	 sal_max: 139170
department: IT/IS 	 sal_max: 139170
department: IT/IS 	 sal_max: 139170
department: IT/IS 	 sal_max: 139170
department: IT/IS 	 sal_max: 120451
department: IT/IS 	 sal_max: 120451
department: IT/IS 	 sal_max: 120451
department: IT/IS 	 sal_max: 120451
department: IT/IS 	 sal_max: 120451
department: IT/IS 	 sal_max: 93312


In [17]:
# which department get the minimum salary for their employees? Top 10 min
sql_sal12 = """
    SELECT department, MIN("Salary Min") as sal_min 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal12)
for row in cursor:
    print('department: %s \t sal_min: %s' %row)

department: Production        	 sal_min: 30000
department: Production        	 sal_min: 30000
department: Production        	 sal_min: 30000
department: Production        	 sal_min: 30000
department: Production        	 sal_min: 30000
department: Production        	 sal_min: 30000
department: Production        	 sal_min: 30000
department: Production        	 sal_min: 30000
department: Production        	 sal_min: 30000
department: Production        	 sal_min: 30000


In [18]:
# which department get the maximum salary for their employees? Top 10 max
sql_sal13 = """
    SELECT "Employee Source", MAX("Salary Max") as sal_max 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal13)
for row in cursor:
    print('Employee Source: %s \t sal_max: %s' %row)

Employee Source: Vendor Referral 	 sal_max: 139170
Employee Source: Vendor Referral 	 sal_max: 139170
Employee Source: Company Intranet - Partner 	 sal_max: 139170
Employee Source: Vendor Referral 	 sal_max: 139170
Employee Source: Employee Referral 	 sal_max: 120451
Employee Source: On-campus Recruiting 	 sal_max: 120451
Employee Source: Vendor Referral 	 sal_max: 120451
Employee Source: Employee Referral 	 sal_max: 120451
Employee Source: Employee Referral 	 sal_max: 120451
Employee Source: Search Engine - Google Bing Yahoo 	 sal_max: 93312


In [19]:
# which department get the minimum salary for their employees? Top 10 min
sql_sal14 = """
    SELECT "Employee Source", MIN("Salary Min") as sal_min 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal14)
for row in cursor:
    print('Employee Source: %s \t sal_min: %s' %row)

Employee Source: Billboard 	 sal_min: 30000
Employee Source: Internet Search 	 sal_min: 30000
Employee Source: Diversity Job Fair 	 sal_min: 30000
Employee Source: Word of Mouth 	 sal_min: 30000
Employee Source: Diversity Job Fair 	 sal_min: 30000
Employee Source: Website Banner Ads 	 sal_min: 30000
Employee Source: Billboard 	 sal_min: 30000
Employee Source: Employee Referral 	 sal_min: 30000
Employee Source: On-campus Recruiting 	 sal_min: 30000
Employee Source: Diversity Job Fair 	 sal_min: 30000


In [5]:
# which sex get the maximum salary for their employees? Top 10 max
sql_sal15 = """
    SELECT sex, MAX("Salary Max") as sal_max 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal15)
for row in cursor:
    print('Sex: %s \t sal_max: %s' %row)

Sex: Male 	 sal_max: 139170
Sex: Female 	 sal_max: 139170
Sex: Male 	 sal_max: 139170
Sex: Female 	 sal_max: 139170
Sex: Male 	 sal_max: 120451
Sex: Male 	 sal_max: 120451
Sex: Female 	 sal_max: 120451
Sex: Male 	 sal_max: 120451
Sex: Female 	 sal_max: 120451
Sex: Female 	 sal_max: 93312


In [6]:
# which sex get the minimum salary for their employees? Top 10 min
sql_sal16 = """
    SELECT sex, MIN("Salary Min") as sal_min 
    FROM hr_dataset JOIN salary_grid ON hr_dataset.position = salary_grid."Position"
    GROUP BY hr_dataset.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal16)
for row in cursor:
    print('Sex: %s \t sal_min: %s' %row)

Sex: Female 	 sal_min: 30000
Sex: Female 	 sal_min: 30000
Sex: Male 	 sal_min: 30000
Sex: Female 	 sal_min: 30000
Sex: Female 	 sal_min: 30000
Sex: Female 	 sal_min: 30000
Sex: Female 	 sal_min: 30000
Sex: Male 	 sal_min: 30000
Sex: Female 	 sal_min: 30000
Sex: Female 	 sal_min: 30000


In [20]:
# 'HR_DATASET' AND 'RECRUITING_COSTS'


In [16]:
# Amount of costs for each month (April, August, December - TOP-3)

sql_recr = """
    (SELECT SUM("January ") FROM public.recruiting_costs) 
    UNION (SELECT SUM("February") FROM public.recruiting_costs) 
    UNION (SELECT SUM("March") FROM public.recruiting_costs) 
    UNION (SELECT SUM("April") FROM public.recruiting_costs) 
    UNION (SELECT SUM("May") FROM public.recruiting_costs) 
    UNION (SELECT SUM("June") FROM public.recruiting_costs) 
    UNION (SELECT SUM("July") FROM public.recruiting_costs) 
    UNION (SELECT SUM("August") FROM public.recruiting_costs) 
    UNION (SELECT SUM("September") FROM public.recruiting_costs) 
    UNION (SELECT SUM("October") FROM public.recruiting_costs) 
    UNION (SELECT SUM("November") FROM public.recruiting_costs) 
    UNION (SELECT SUM("December") FROM public.recruiting_costs);
"""

cursor.execute(sql_recr)
for row in cursor:
    print(row[0])

6497
6584
4389
11177
5881
5209
6616
10032
7278
5652
6343
8807


In [30]:
# From what sources do we find the longest-serving employees?
sql_recr2 = """
    SELECT "Employee Source" FROM public.hr_dataset 
    ORDER BY "Days Employed" DESC LIMIT 10;
"""

cursor.execute(sql_recr2)
for row in cursor:
    print(row[0])

Billboard
Search Engine - Google Bing Yahoo
Billboard
Diversity Job Fair
Other
Other
Billboard
Diversity Job Fair
Employee Referral
Search Engine - Google Bing Yahoo


In [31]:
# From what sources do we find the most under-employed employees?
sql_recr3 = """
    SELECT "Employee Source" FROM public.hr_dataset 
    ORDER BY "Days Employed" ASC LIMIT 10;
"""

cursor.execute(sql_recr3)
for row in cursor:
    print(row[0])

Diversity Job Fair
Pay Per Click
Employee Referral
Word of Mouth
Diversity Job Fair
Social Networks - Facebook Twitter etc
Search Engine - Google Bing Yahoo
Glassdoor
Newspager/Magazine
Word of Mouth


In [6]:
# II. LET'S EXPLORE THE TABLE 'PRODUCTION_STAFF'!


In [9]:
# Is there a relationship between performance score and who the employee is working under?
# - Fully Meets
sql_str64 = """
    SELECT "Manager Name" FROM public.production_staff 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str64)

{'Elisa Bramante': 10, 'Elijiah Gray': 14, 'Brannon Miller': 8, 'Kelley Spirea': 15, 'Michael Albert': 10, 'Webster Butler': 11, 'Amy Dunn': 11, 'Ketsia Liebig': 14, 'David Stanley': 15, 'Kissy Sullivan': 14}


In [10]:
# Is there a relationship between performance score and who the employee is working under?
# - Exceeds
sql_str65 = """
    SELECT "Manager Name" FROM public.production_staff 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str65)

{'Elisa Bramante': 2, 'Michael Albert': 2, 'Brannon Miller': 5, 'Amy Dunn': 2, 'Webster Butler': 2, 'Kelley Spirea': 3, 'David Stanley': 1, 'Kissy Sullivan': 1, 'Ketsia Liebig': 2, 'Elijiah Gray': 2}


In [11]:
# Is there a relationship between performance score and who the employee is working under?
# - Needs Improvement
sql_str66 = """
    SELECT "Manager Name" FROM public.production_staff 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str66)

{'Elisa Bramante': 2, 'Kissy Sullivan': 1, 'Brannon Miller': 1, 'Michael Albert': 3, 'Webster Butler': 2, 'Ketsia Liebig': 1, 'Elijiah Gray': 1}


In [16]:
# Is there a relationship between performance score and the race of the employee?
# - Fully Meets
sql_str67 = """
    SELECT "Race Desc" FROM public.production_staff 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str67)

{'White': 80, 'Asian': 14, 'Hispanic': 2, 'Black or African American': 20, 'Two or more races': 5, 'American Indian or Alaska Native': 1}


In [17]:
# Is there a relationship between performance score and the race of the employee?
# - Exceeds
sql_str68 = """
    SELECT "Race Desc" FROM public.production_staff 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str68)

{'White': 15, 'Two or more races': 2, 'American Indian or Alaska Native': 2, 'Asian': 2, 'Black or African American': 1}


In [18]:
# Is there a relationship between performance score and the race of the employee?
# - Needs Improvement
sql_str69 = """
    SELECT "Race Desc" FROM public.production_staff 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str69)

{'Asian': 1, 'White': 5, 'Black or African American': 5}


In [19]:
# Is there a relationship between performance score and the department of the employee?
# - Fully Meets
sql_str70 = """
    SELECT "Department" FROM public.production_staff 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str70)

{'Production       ': 122}


In [20]:
# Is there a relationship between performance score and the department of the employee?
# - Exceeds
sql_str71 = """
    SELECT "Department" FROM public.production_staff 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str71)

{'Production       ': 22}


In [21]:
# Is there a relationship between performance score and the department of the employee?
# - Needs Improvement
sql_str72 = """
    SELECT "Department" FROM public.production_staff 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str72)

{'Production       ': 11}


In [25]:
# Is there a relationship between performance score and the position of the employee?
# - Fully Meets
sql_str73 = """
    SELECT "Position" FROM public.production_staff 
    WHERE "Performance Score" = 'Fully Meets';
"""
sql_medium(sql_str73)

{'Production Manager': 10, 'Production Technician I': 79, 'Production Technician II': 33}


In [26]:
# Is there a relationship between performance score and the position of the employee?
# - Exceeds
sql_str74 = """
    SELECT "Position" FROM public.production_staff 
    WHERE "Performance Score" = 'Exceeds';
"""
sql_medium(sql_str74)

{'Production Manager': 2, 'Production Technician I': 14, 'Production Technician II': 6}


In [27]:
# Is there a relationship between performance score and the position of the employee?
# - Needs Improvement
sql_str75 = """
    SELECT "Position" FROM public.production_staff 
    WHERE "Performance Score" = 'Needs Improvement';
"""
sql_medium(sql_str75)

{'Production Manager': 2, 'Production Technician I': 8, 'Production Technician II': 1}


In [ ]:
# 'PRODUCTION_STAFF' AND 'SALARY_GRID'


In [32]:
# which managers get the maximum salary for their employees? Top 10 max
sql_sal17 = """
    SELECT production_staff."Manager Name", MAX("Salary Max") as sal_max 
    FROM production_staff JOIN salary_grid ON production_staff."Position" = salary_grid."Position"
    GROUP BY production_staff.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal17)
for row in cursor:
    print('manage_name: %s \t sal_max: %s' %row)

manage_name: Michael Albert 	 sal_max: 58000
manage_name: Kissy Sullivan 	 sal_max: 58000
manage_name: Webster Butler 	 sal_max: 58000
manage_name: David Stanley 	 sal_max: 58000
manage_name: Kelley Spirea 	 sal_max: 58000
manage_name: Amy Dunn 	 sal_max: 58000
manage_name: David Stanley 	 sal_max: 58000
manage_name: Webster Butler 	 sal_max: 58000
manage_name: Kissy Sullivan 	 sal_max: 58000
manage_name: Kissy Sullivan 	 sal_max: 58000


In [33]:
# which managers get the minimum salary for their employees? Top 10 min
sql_sal18 = """
    SELECT production_staff."Manager Name", MIN("Salary Min") as sal_min 
    FROM production_staff JOIN salary_grid ON production_staff."Position" = salary_grid."Position"
    GROUP BY production_staff.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal18)
for row in cursor:
    print('manage_name: %s \t sal_min: %s' %row)

manage_name: Kelley Spirea 	 sal_min: 30000
manage_name: Amy Dunn 	 sal_min: 30000
manage_name: Kelley Spirea 	 sal_min: 30000
manage_name: Kissy Sullivan 	 sal_min: 30000
manage_name: David Stanley 	 sal_min: 30000
manage_name: Ketsia Liebig 	 sal_min: 30000
manage_name: Kelley Spirea 	 sal_min: 30000
manage_name: Webster Butler 	 sal_min: 30000
manage_name: Michael Albert 	 sal_min: 30000
manage_name: Ketsia Liebig 	 sal_min: 30000


In [34]:
# which racedesc get the maximum salary for their employees? Top 10 max
sql_sal19 = """
    SELECT production_staff."Race Desc", MAX("Salary Max") as sal_max 
    FROM production_staff JOIN salary_grid ON production_staff."Position" = salary_grid."Position"
    GROUP BY production_staff.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal19)
for row in cursor:
    print('race_desc: %s \t sal_max: %s' %row)

race_desc: White 	 sal_max: 58000
race_desc: Black or African American 	 sal_max: 58000
race_desc: White 	 sal_max: 58000
race_desc: White 	 sal_max: 58000
race_desc: Hispanic 	 sal_max: 58000
race_desc: White 	 sal_max: 58000
race_desc: White 	 sal_max: 58000
race_desc: White 	 sal_max: 58000
race_desc: White 	 sal_max: 58000
race_desc: Two or more races 	 sal_max: 58000


In [36]:
# which racedesc get the minimum salary for their employees? Top 10 min
sql_sal20 = """
    SELECT production_staff."Race Desc", MIN("Salary Min") as sal_min 
    FROM production_staff JOIN salary_grid ON production_staff."Position" = salary_grid."Position"
    GROUP BY production_staff.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal20)
for row in cursor:
    print('race_desc: %s \t sal_min: %s' %row)

race_desc: Asian 	 sal_min: 30000
race_desc: White 	 sal_min: 30000
race_desc: White 	 sal_min: 30000
race_desc: Asian 	 sal_min: 30000
race_desc: White 	 sal_min: 30000
race_desc: Two or more races 	 sal_min: 30000
race_desc: White 	 sal_min: 30000
race_desc: Two or more races 	 sal_min: 30000
race_desc: White 	 sal_min: 30000
race_desc: White 	 sal_min: 30000


In [37]:
# which position get the maximum salary for their employees? Top 10 max
sql_sal21 = """
    SELECT production_staff."Position", MAX("Salary Max") as sal_max 
    FROM production_staff JOIN salary_grid ON production_staff."Position" = salary_grid."Position"
    GROUP BY production_staff.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal21)
for row in cursor:
    print('position: %s \t sal_max: %s' %row)

position: Production Technician II 	 sal_max: 58000
position: Production Technician II 	 sal_max: 58000
position: Production Technician II 	 sal_max: 58000
position: Production Technician II 	 sal_max: 58000
position: Production Technician II 	 sal_max: 58000
position: Production Technician II 	 sal_max: 58000
position: Production Technician II 	 sal_max: 58000
position: Production Technician II 	 sal_max: 58000
position: Production Technician II 	 sal_max: 58000
position: Production Technician II 	 sal_max: 58000


In [39]:
# which position get the minimum salary for their employees? Top 10 min
sql_sal22 = """
    SELECT production_staff."Position", MIN("Salary Min") as sal_min 
    FROM production_staff JOIN salary_grid ON production_staff."Position" = salary_grid."Position"
    GROUP BY production_staff.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal22)
for row in cursor:
    print('position: %s \t sal_min: %s' %row)

position: Production Technician I 	 sal_min: 30000
position: Production Technician I 	 sal_min: 30000
position: Production Technician I 	 sal_min: 30000
position: Production Technician I 	 sal_min: 30000
position: Production Technician I 	 sal_min: 30000
position: Production Technician I 	 sal_min: 30000
position: Production Technician I 	 sal_min: 30000
position: Production Technician I 	 sal_min: 30000
position: Production Technician I 	 sal_min: 30000
position: Production Technician I 	 sal_min: 30000


In [40]:
# which employment status get the maximum salary for their employees? Top 10 max
sql_sal23 = """
    SELECT production_staff."Employment Status", MAX("Salary Max") as sal_max 
    FROM production_staff JOIN salary_grid ON production_staff."Position" = salary_grid."Position"
    GROUP BY production_staff.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal23)
for row in cursor:
    print('empl_stat: %s \t sal_max: %s' %row)

empl_stat: Voluntarily Terminated 	 sal_max: 58000
empl_stat: Active 	 sal_max: 58000
empl_stat: Active 	 sal_max: 58000
empl_stat: Voluntarily Terminated 	 sal_max: 58000
empl_stat: Leave of Absence 	 sal_max: 58000
empl_stat: Voluntarily Terminated 	 sal_max: 58000
empl_stat: Leave of Absence 	 sal_max: 58000
empl_stat: Future Start 	 sal_max: 58000
empl_stat: Voluntarily Terminated 	 sal_max: 58000
empl_stat: Active 	 sal_max: 58000


In [42]:
# which employment status get the minimum salary for their employees? Top 10 min
sql_sal24 = """
    SELECT production_staff."Employment Status", MIN("Salary Min") as sal_min 
    FROM production_staff JOIN salary_grid ON production_staff."Position" = salary_grid."Position"
    GROUP BY production_staff.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal24)
for row in cursor:
    print('empl_stat: %s \t sal_min: %s' %row)

empl_stat: Active 	 sal_min: 30000
empl_stat: Active 	 sal_min: 30000
empl_stat: Active 	 sal_min: 30000
empl_stat: Voluntarily Terminated 	 sal_min: 30000
empl_stat: Active 	 sal_min: 30000
empl_stat: Voluntarily Terminated 	 sal_min: 30000
empl_stat: Active 	 sal_min: 30000
empl_stat: Voluntarily Terminated 	 sal_min: 30000
empl_stat: Active 	 sal_min: 30000
empl_stat: Leave of Absence 	 sal_min: 30000


In [43]:
# which performance score get the maximum salary for their employees? Top 10 max
sql_sal25 = """
    SELECT production_staff."Performance Score", MAX("Salary Max") as sal_max 
    FROM production_staff JOIN salary_grid ON production_staff."Position" = salary_grid."Position"
    GROUP BY production_staff.id ORDER BY sal_max DESC LIMIT 10;
"""

cursor.execute(sql_sal25)
for row in cursor:
    print('perf_score: %s \t sal_max: %s' %row)

perf_score: Fully Meets 	 sal_max: 58000
perf_score: Exceptional 	 sal_max: 58000
perf_score: Fully Meets 	 sal_max: 58000
perf_score: N/A- too early to review 	 sal_max: 58000
perf_score: Fully Meets 	 sal_max: 58000
perf_score: Fully Meets 	 sal_max: 58000
perf_score: Fully Meets 	 sal_max: 58000
perf_score: N/A- too early to review 	 sal_max: 58000
perf_score: Fully Meets 	 sal_max: 58000
perf_score: 90-day meets 	 sal_max: 58000


In [18]:
# which performance score get the minimum salary for their employees? Top 10 max
sql_sal26 = """
    SELECT production_staff."Performance Score", MIN("Salary Min") as sal_min 
    FROM production_staff JOIN salary_grid ON production_staff."Position" = salary_grid."Position"
    GROUP BY production_staff.id ORDER BY sal_min ASC LIMIT 10;
"""

cursor.execute(sql_sal26)
for row in cursor:
    print('perf_score: %s \t sal_min: %s' %row)

perf_score: Fully Meets 	 sal_min: 30000
perf_score: Exceeds 	 sal_min: 30000
perf_score: Fully Meets 	 sal_min: 30000
perf_score: 90-day meets 	 sal_min: 30000
perf_score: Fully Meets 	 sal_min: 30000
perf_score: Fully Meets 	 sal_min: 30000
perf_score: 90-day meets 	 sal_min: 30000
perf_score: Exceeds 	 sal_min: 30000
perf_score: Fully Meets 	 sal_min: 30000
perf_score: Fully Meets 	 sal_min: 30000


In [2]:
# How many people worked in each department, how many left, and what percentage? 
sql0 = """
    WITH Total AS (SELECT department, COUNT("id") as count_id FROM hr_dataset 
    GROUP BY hr_dataset.department ORDER BY count_id DESC), 
    Terminate AS (SELECT department, COUNT("id") as count_id_t FROM hr_dataset 
    WHERE "Employment Status" NOT IN ('Future Start', 
    'Active', 'Leave of Absence') GROUP BY hr_dataset.department)
    SELECT A.*, Terminate.count_id_t, (count_id_t*100.00/count_id)
    FROM Total A JOIN Terminate ON 
    A.department = Terminate.department;
"""

cursor.execute(sql0)
for row in cursor:
    print(row)

('Production       ', 208, 83, Decimal('39.9038461538461538'))
('IT/IS', 50, 10, Decimal('20.0000000000000000'))
('Sales', 31, 4, Decimal('12.9032258064516129'))
('Admin Offices', 10, 2, Decimal('20.0000000000000000'))
('Software Engineering', 10, 3, Decimal('30.0000000000000000'))




#### Таблица hr_dataset

Имеется зависимость между оценкой производительности («полностью соответствует») и тем, под чьим руководством работает сотрудник, наибольшее число таких сотрудников обнаружено у 4-х менеджеров – Kelley Spirea, Ketsia Liebig, David Stanley, Kissy Sullivan. 

Имеется зависимость между оценкой производительности («полностью соответствует» и «превышает») и семейным положением сотрудника – в основном это одинокие люди, на втором месте идут семейные.

Ярко выраженной зависимости между оценкой производительности («полностью соответствует») и возрастом сотрудника не имеется, но у наибольшего количества сотрудников зафиксирован возраст 34 и 39 лет. 

Имеется зависимость между оценкой производительности («полностью соответствует» и «превышает») и гражданством сотрудника – в основном это граждане США. 

В основном, положительная оценка производительности («полностью соответствует» и «превышает») наблюдается в департаменте производства. 

С большим отрывом от остальных должностей показывают положительную оценку производительности («полностью соответствует» и «превышает») сотрудники должностей Production Technician I и  Production Technician II. 

Имеется зависимость между положительной оценкой производительности («полностью соответствует» и «превышает») и источником рекрутинга, откуда нашли специалистов – Diversity Job Fair, Monster.com, Search Engine - Google Bing Yahoo, Employee Referral. Т.е. наиболее производительных сотрудников вероятнее всего нужно искать на этих источниках.

Наиболее продолжительное время работают сотрудники: 
- замужние/женатые; 
- с гражданством США; 
- департамента производства;
- на должности Production Technician I;
- рекрутированные через следующие источники – Billboard', 'Search Engine - Google Bing Yahoo', 'Diversity Job Fair' (скорее всего сотрудников, которые будут работать продолжительное время следует искать через эти источники); 
- под руководством Janet King.

В основном среди тех, кто уже уволился, имеются сотрудники, которые: 
- работали под руководством Webster Butler, Kissy Sullivan, Kissy Sullivan (необходимо понять, почему так происходит у данных менеджеров); 
- замужние/женатые; 
- с гражданством США;
- из департамента производства; 
- на должности Production Technician I и Production Technician II; 
- имели оценку производительности соответствующую стандартам. 

Максимальную зарплату получали сотрудники (топ-10): 
- под руководством Simon Roup и Peter Monroe; 
- с гражданством США; 
- одинокие и замужние/женатые; 
- департамента IT/IS; 
- найденные через Vendor Referral и Employee Referral; 
- мужского и женского пола (распределение среди топ-10 – 50 на 50%). 

Минимальную зарплату получали сотрудники (топ-10): 
- под руководством Webster Butler, David Stanley и Brannon Miller; 
- с гражданством США; 
- в возрасте 43, 49 и 51 года;
- департамента Production; 
- найденные через Diversity Job Fair и Billboard;
- в основном женского пола.

По количеству сотрудников женщин получается больше чем мужчин (женщины – 177 человек, мужчины – 133 человека).


#### Таблица recruiting_costs

Среди всех 12 месяцев наибольшие затраты на рекрутинг сложились за апрель, август и декабрь.

Больше всего за год потрачено на MBTA ads, Diversity Job Fair и Newspager/Magazine.


#### Таблица production_staff

В данной таблице приведены данные по департаменту Production. 

Имеется зависимость между оценкой производительности («полностью соответствует») и тем, под чьим руководством работает сотрудник, наибольшее число таких сотрудников обнаружено у 5-ти менеджеров – Kelley Spirea, Ketsia Liebig, David Stanley, Kissy Sullivan и Elijiah Gray. 

С большим отрывом от остальных должностей показывают положительную оценку производительности («полностью соответствует» и «превышает») сотрудники на должности Production Technician I. 

Максимальную зарплату получали сотрудники (топ-10): 
- под руководством Kissy Sullivan, Webster Butler и David Stanley; 
- на должности Production Technician II. 

Минимальную зарплату получали сотрудники (топ-10): 
- под руководством Kelley Spirea и Ketsia Liebig; 
- Production Technician I. 

